In [1]:
import numpy as np
import pandas as pd
import random
import sklearn

In [2]:
import sys
sys.path.append('..')


In [5]:
from feature_extract import *
from utils import load_data
from models.lgb_model import lgb_model_for_upload,lgb_model_for_offline_test

In [6]:
import os
path0=os.path.abspath(os.path.dirname(os.getcwd()))
path = path0 +'/data/info_dataset/'
path1 = path0+'/data/'

In [7]:
# get feature

In [8]:
person =person_feature(path)
intent_table = person_intent(path)
work_history, work_history_table = work_history(path)
cert_table = certificate(path)
project_history, project_history_table = project_history(path)
job_table = job_table(path)

In [9]:
# load data
train_data, valid_data, test_data =load_data(path1)
del train_data['Unnamed: 0']
del valid_data['Unnamed: 0']
del test_data['Unnamed: 0']

In [10]:
data_all = pd.concat([train_data,valid_data,test_data], ignore_index=True)
candidate_data, job_data = diliver_current(data_all)

## offline

In [11]:
train=get_k_fold_data(train_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
valid_data0 =data_merge(valid_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
valid = get_data(valid_data0, train)


In [10]:
from match_func import *

In [12]:
person.head()

,求职者编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,语言能力,专业特长
0,33291,1,15,3.0,计算机应用技术,37,网络管理/信息安全管理,NaN,深圳市,普通话：精通 广东话：精通******语：听：良好，说：良好，读：良好，写：,"1、精通计算机软、硬件及网络维护，能迅速处理各种突发的计算机故障 2、熟悉局域网建设和管理,..."
1,2985277,1,12,3.0,计算机应用技术,35,*公关/营销/业务类,文化体育行业,深圳市,普通话：精通 广东话：精通************语：听：精通，说：精通，读,..
2,2982066,0,10,3.0,金融学含保险学,32,出纳,医药销售行业,南山区,普通话：精通 广东话：精通************语：听：精通，说：良好，读,****-****年获校三等奖学金****-****年获单项奖学金
3,3010866,1,10,1.0,物理电子学,34,营销代表/销售顾问,珠宝玉石行业,深圳市,普通话：精通 广东话：良好************语：听：一般，说：不会，读,"我对塑胶厂里的运作流程非常熟悉,从业务,计划,生产,人事,采购,客户服务都有经验."
4,316816964,0,15,1.0,学前教育学,34,小学教育/幼儿教育/保育,行业组织,福田区,NaN,1：拥有丰富的办公室工作经历，能独挡一面处理工作中的相关事情。2：熟练掌握Windows办公...


In [13]:
import time
for func in tqdm(["simhash", "sim_jaccard", "sim_edit", "difflibs"]):
    st =time.time()

    person[f"应聘者专业_最近工作岗位_{func}"]  = person.apply(lambda x: eval(func)(x['应聘者专业'], x['最近工作岗位']), axis=1)
    st2 =time.time()
    df_temp1 = person.groupby("求职者编号")[f"应聘者专业_最近工作岗位_{func}"].agg(
                    [(f"应聘者专业_最近工作岗位_{func}_mean", "mean"),
                     (f"应聘者专业_最近工作岗位_{func}_max", "max"),
                     (f"应聘者专业_最近工作岗位_{func}_min", "min"),
                     (f"应聘者专业_最近工作岗位_{func}_std", "std")]
                ).reset_index()
    print(f"应聘者专业_最近工作岗位_{func}_groupby，共耗时:",time.time() - st)


 25%|█████████████████████                                                               | 1/4 [00:22<01:07, 22.54s/it]

应聘者专业_最近工作岗位_simhash_groupby，共耗时: 22.5424587726593



 50%|██████████████████████████████████████████                                          | 2/4 [00:26<00:34, 17.11s/it]

应聘者专业_最近工作岗位_sim_jaccard_groupby，共耗时: 4.415237188339233



 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:44<00:17, 17.35s/it]

应聘者专业_最近工作岗位_sim_edit_groupby，共耗时: 17.906723022460938



100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:50<00:00, 12.59s/it]

应聘者专业_最近工作岗位_difflibs_groupby，共耗时: 5.478602409362793


In [10]:
train_data.groupby("岗位编号").aggregate({"标签": "mean"}).reset_index()

,岗位编号,标签
0,135144,0.6
1,137045,0.0
2,146798,0.0
3,436321,0.0
4,462715,0.5
...,...,...
1838,44274688,1.0
1839,44285651,1.0
1840,44298679,1.0
1841,44310665,1.0


In [12]:
test_data0 =data_merge(test_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
test = get_data(test_data0, train)
temp_data= pd.concat([train,valid, test], ignore_index=True)

In [15]:
def deliver_record_feature(data_feature):
    for f1, f2 in tqdm([["岗位编号", "求职意向岗位类别"], ["岗位编号", "应聘者专业"], ["岗位编号", "求职意向工作地点"]]):
        data_feature[f"{f1}_{f2}_nunique"] = data_feature.groupby(
            [f1])[f2].transform("nunique")
    # 最大值、最小值、均值、标准差、众数、众数占比
    for f1, f2 in tqdm([["岗位编号", "年龄"], ["岗位编号", "工作年限"], ["岗位编号", "工作经历数"], ["岗位编号", "项目经验数"], ["岗位编号", "总主要业绩字数"],
                        ["岗位编号", "最高学历"], ["岗位编号", "可到职天数"]]):
        df_temp1 = data_feature.groupby(f1)[f2].agg(
            [(f"{f1}_{f2}_mean", "mean"),
             (f"{f1}_{f2}_max", "max"),
             (f"{f1}_{f2}_min", "min"),
             (f"{f1}_{f2}_std", "std"),
             (f"{f1}_{f2}_median", np.median),
             (f"{f1}_{f2}_mode", mode),
             (f"{f1}_{f2}_mode_rate", mode_rate)]
        ).reset_index()
        data_feature = data_feature.merge(df_temp1, how="left")

    # 男性占比、女性个数占比、工作地点符合否占比。
    for f1, f2 in [["岗位编号", "性别"], ["岗位编号", "工作地点符合否"]]:  # 性别众数及占比，工作地点符合及占比
        df_temp2 = data_feature.groupby(f1)[f2].agg(
            [
                (f"{f1}_{f2}_mode", mode),
                (f"{f1}_{f2}_mode_rate", mode_rate)]
        ).reset_index()
        data_feature = data_feature.merge(df_temp2, how="left")
    return data_feature

In [153]:
temp_data=deliver_record_feature(temp_data)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:05<00:00,  1.22it/s]


In [154]:
temp_data

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106060,44350657,321173912,0,1,5,4.0,377,26,198,38,...,180.0,7.0,56.303740,7.0,7.0,0.555556,1,0.888889,1.0,1.000000
106061,44350657,321377962,0,1,0,4.0,292,28,226,0,...,180.0,7.0,56.303740,7.0,7.0,0.555556,1,0.888889,1.0,1.000000
106062,47578782,319583916,0,0,0,4.0,302,28,226,0,...,7.0,7.0,NaN,7.0,7.0,1.000000,0,1.000000,0.0,1.000000
106063,47649893,6079047,0,1,7,4.0,0,29,174,82,...,7.0,7.0,0.000000,7.0,7.0,1.000000,1,1.000000,0.0,1.000000


In [17]:
temp_data[:1].values

array([[775040, 5969825, 0, 1, 12, 3.0, None, 34, '品质/质检', nan, '深圳市',
        '外语语种： 语    外语水平： 一般 ************ ******',
        '1.对五金的C机加,冲压、数冲/折弯/焊接/研磨、拉丝/涂装/装配，等各制程的工艺流程，品质的管理和改善提高熟悉。2.熟悉塑胶的成型/二次加工，对其品质管理,分析改善有一定的经验。3.熟悉光电类的光纤连接线和适配器的生产工艺和质量的管控熟悉。 4.现场的7S管理，员工的培训/考核以及执行力与凝聚力的加强有一定的经验。5.熟悉ISO****质量管理体系和相关品质管理工具,如QC七大手法, 8D等。6.能独立编写品质手册以及相关程序文件，能制做相关多媒体科教资料。7.有组织内审，接洽顾客审厂、第三方审核的经验。',
        2.0, '社会无职', 'IPQC/组长', nan, 2.0, '深圳市', 0,
        '熟悉电脑办公软件运用及多项品质管理手法，认识或了解变压器制程及工艺要求，具有强烈的品质观念。', 46.0, False,
        '一直以来，以以身作则，不断进取的心态去面对工作和生活，在带给别人快乐和收获的同时，也陶冶，充实自己。', '品质质检',
        '深圳市', '其它', 7.0, '品质工程师', 50.0, nan, 4.0, 161.25, 645.0, 8, 88,
        0.5303030303030303, 1.0, 0.4545454545454546, 0.5606060606060606,
        0.5151515151515151, 0.5076923076923077, 0.5303030303030303, 1.0,
        0.4545454545454546, 0.6363636363636364, 0.4696969696969697,
        0.5909090909090908, 0.5606060606060606, 0.5151515151515151,
        0.5076923076923077, 0.4545454545454546, 0.5, 0.4615384

In [12]:
temp_data.shape

(106065, 113)

In [20]:
pd.DataFrame(list(temp_data.columns)).to_csv(r'columns.csv')

In [155]:
from sklearn.preprocessing import LabelEncoder

for f in temp_data.select_dtypes('object'):
    le = LabelEncoder()
    print(f)
    temp_data[f] = le.fit_transform(temp_data[f].astype('str'))
temp_data["招聘对象"] =le.fit_transform(temp_data["招聘对象"].astype('str'))
temp_data["最近工作岗位"] =le.fit_transform(temp_data["最近工作岗位"].astype('str'))
temp_data["求职意向岗位类别"] =le.fit_transform(temp_data["求职意向岗位类别"].astype('str'))

In [30]:
# onehot
data_all_feature=feature_process(temp_data)

In [156]:
# none one-hot
data_all_feature=temp_data

In [25]:
data_all_feature.head(1)

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364


In [21]:
train.shape,valid.shape,test.shape

((28232, 120), (7059, 120), (70774, 120))

In [157]:
train0 = data_all_feature[:train.shape[0]]#.标签.value_counts()
valid0 = data_all_feature[train.shape[0]:train.shape[0]+valid.shape[0]]
test0 = data_all_feature[train.shape[0]+valid.shape[0]:]
train_label = train0[["标签"]]
valid_label = valid0[["标签"]]
train=train0[list(train0.columns)[:2]+list(train0.columns)[3:]]
valid=valid0[list(train0.columns)[:2]+list(train0.columns)[3:]]


# k-fold offline 分析特征的有效性

In [158]:
import lightgbm
from sklearn.metrics import f1_score
params = {"objective": "binary",
          "learning_rate": 0.05,
          "max_depth": 6,
          "num_leaves": 32,
          "verbose": -1,
          "bagging_fraction": 0.8,
          "feature_fraction": 0.9,
          'subsample':0.85,
          'bagging_freq':1,
          'random_state':2048}
def lgb_model_for_offline_test(train, train_label, valid, valid_label):
    train_set = lightgbm.Dataset(train.values, label=train_label)
    val_set = lightgbm.Dataset(valid.values, label=valid_label)
    
    model = lightgbm.train(train_set=train_set,
                           valid_sets=[val_set],
                           num_boost_round=3000,
                           params=params,
                           feval=lgb_f1_score,
                           early_stopping_rounds=100)
    return model
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)  # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [170]:
train

,岗位编号,求职者编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,1,12,3.0,0,34,203,0,43,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,1,12,4.0,216,37,283,116,43,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,1,0,1.0,276,31,152,0,43,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,6,4.0,373,33,142,0,43,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,3.0,349,29,24,0,43,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28227,800360,1633998,0,18,3.0,391,40,158,28,43,...,180.0,7.0,19.838038,7.0,7.0,0.879518,0,0.658824,0.0,0.964706
28228,43922680,5733578,0,10,2.0,363,28,84,82,43,...,180.0,7.0,33.637860,7.0,7.0,0.822511,0,0.974249,1.0,0.751073
28229,816862,6088168,1,0,4.0,0,30,143,93,19,...,180.0,7.0,16.169290,7.0,7.0,0.909836,1,0.894309,0.0,0.951220
28230,772339,4684178,1,11,3.0,0,34,1,60,43,...,180.0,7.0,20.659317,7.0,7.0,0.902439,1,0.772414,0.0,0.975862


In [167]:
import lightgbm
from sklearn.model_selection import KFold
X = pd.concat([train,valid],ignore_index=True).reset_index(drop=True)
y = pd.concat([train_label,valid_label],ignore_index=True).reset_index(drop=True)
oof = []
prediction = test[['岗位编号', '求职者编号']]
prediction['pred'] = 0
df_importance_list = []
# model = lightgbm.LGBMClassifier(num_leaves=64,
#                            max_depth=10,
#                            learning_rate=0.1,
#                            n_estimators=1000000,
#                            subsample=0.8,
#                            feature_fraction=0.8,
#                            reg_alpha=0.5,
#                            reg_lambda=0.5,
#                            random_state=2048,
#                            metric='auc')
kfold = KFold(n_splits=5, random_state=2048, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(X, y)):
    X_train = X.loc[trn_idx]
    Y_train = y.loc[trn_idx]

    X_val = X.loc[val_idx]
    Y_val = y.loc[val_idx]
    train_set = lightgbm.Dataset(X_train.values, label=Y_train)
    val_set = lightgbm.Dataset(X_val.values, label=Y_val)
    
    lgb_model = lightgbm.train(train_set=train_set,
                           valid_sets=[val_set],
                           num_boost_round=3000,
                           params=params,
                           feval=lgb_f1_score,
                           early_stopping_rounds=100)
    print('\nFold_{} Training ================================\n'.format(fold_id + 1))

    pred_val = lgb_model.predict(X_val)
    df_oof = X.iloc[val_idx][['岗位编号', '求职者编号']].copy()
    df_oof['标签'] = y.iloc[val_idx][['标签']].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict(test0[list(test0.columns)[:2]+list(test0.columns)[3:]].values)
    prediction['pred'] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': list(test0.columns)[:2]+list(test0.columns)[3:],
        'importance': lgb_model.feature_importance(),
    })
    df_importance_list.append(df_importance)


d:\software\python\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


[1]	valid_0's binary_logloss: 0.409228	valid_0's f1: 0
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.380412	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.356689	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.336876	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.319829	valid_0's f1: 0
[6]	valid_0's binary_logloss: 0.30441	valid_0's f1: 0
[7]	valid_0's binary_logloss: 0.290667	valid_0's f1: 0
[8]	valid_0's binary_logloss: 0.278264	valid_0's f1: 0
[9]	valid_0's binary_logloss: 0.267246	valid_0's f1: 0
[10]	valid_0's binary_logloss: 0.257247	valid_0's f1: 0.675708
[11]	valid_0's binary_logloss: 0.250106	valid_0's f1: 0.701329
[12]	valid_0's binary_logloss: 0.241538	valid_0's f1: 0.733521
[13]	valid_0's binary_logloss: 0.233599	valid_0's f1: 0.76075
[14]	valid_0's binary_logloss: 0.226262	valid_0's f1: 0.779643
[15]	valid_0's binary_logloss: 0.219378	valid_0's f1: 0.803365
[16]	valid_0's binary_logloss: 0.212969	valid_0's f1: 0.810642
[17

[134]	valid_0's binary_logloss: 0.110931	valid_0's f1: 0.865042
[135]	valid_0's binary_logloss: 0.110973	valid_0's f1: 0.864637
[136]	valid_0's binary_logloss: 0.110961	valid_0's f1: 0.866105
[137]	valid_0's binary_logloss: 0.111021	valid_0's f1: 0.865295
[138]	valid_0's binary_logloss: 0.110891	valid_0's f1: 0.86623
[139]	valid_0's binary_logloss: 0.110819	valid_0's f1: 0.865016
[140]	valid_0's binary_logloss: 0.110795	valid_0's f1: 0.864486
[141]	valid_0's binary_logloss: 0.110711	valid_0's f1: 0.863955
[142]	valid_0's binary_logloss: 0.110698	valid_0's f1: 0.864359
[143]	valid_0's binary_logloss: 0.110717	valid_0's f1: 0.864359
[144]	valid_0's binary_logloss: 0.110698	valid_0's f1: 0.863955
[145]	valid_0's binary_logloss: 0.110705	valid_0's f1: 0.863551
[146]	valid_0's binary_logloss: 0.110707	valid_0's f1: 0.863551
[147]	valid_0's binary_logloss: 0.110713	valid_0's f1: 0.863551
[148]	valid_0's binary_logloss: 0.110682	valid_0's f1: 0.864082
[149]	valid_0's binary_logloss: 0.11069	v

[264]	valid_0's binary_logloss: 0.110408	valid_0's f1: 0.865142
[265]	valid_0's binary_logloss: 0.110336	valid_0's f1: 0.865142
[266]	valid_0's binary_logloss: 0.110359	valid_0's f1: 0.866635
[267]	valid_0's binary_logloss: 0.110385	valid_0's f1: 0.86729
[268]	valid_0's binary_logloss: 0.110377	valid_0's f1: 0.86729
[269]	valid_0's binary_logloss: 0.110353	valid_0's f1: 0.86595
[270]	valid_0's binary_logloss: 0.110392	valid_0's f1: 0.866885
[271]	valid_0's binary_logloss: 0.110429	valid_0's f1: 0.866355
[272]	valid_0's binary_logloss: 0.110393	valid_0's f1: 0.8657
[273]	valid_0's binary_logloss: 0.110435	valid_0's f1: 0.865295
[274]	valid_0's binary_logloss: 0.110447	valid_0's f1: 0.866355
[275]	valid_0's binary_logloss: 0.110456	valid_0's f1: 0.866355
[276]	valid_0's binary_logloss: 0.110488	valid_0's f1: 0.866355
[277]	valid_0's binary_logloss: 0.110564	valid_0's f1: 0.866355
[278]	valid_0's binary_logloss: 0.110559	valid_0's f1: 0.86595
[279]	valid_0's binary_logloss: 0.110535	valid

d:\software\python\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

[268]	valid_0's binary_logloss: 0.108627	valid_0's f1: 0.863636
[269]	valid_0's binary_logloss: 0.108639	valid_0's f1: 0.864045
[270]	valid_0's binary_logloss: 0.108704	valid_0's f1: 0.863098
[271]	valid_0's binary_logloss: 0.108712	valid_0's f1: 0.863098
[272]	valid_0's binary_logloss: 0.108752	valid_0's f1: 0.863507
[273]	valid_0's binary_logloss: 0.108685	valid_0's f1: 0.863917
[274]	valid_0's binary_logloss: 0.108634	valid_0's f1: 0.863098
[275]	valid_0's binary_logloss: 0.108609	valid_0's f1: 0.862968
[276]	valid_0's binary_logloss: 0.108588	valid_0's f1: 0.862838
[277]	valid_0's binary_logloss: 0.10858	valid_0's f1: 0.862838
[278]	valid_0's binary_logloss: 0.108502	valid_0's f1: 0.864455
[279]	valid_0's binary_logloss: 0.108502	valid_0's f1: 0.864326
[280]	valid_0's binary_logloss: 0.108522	valid_0's f1: 0.863378
[281]	valid_0's binary_logloss: 0.108548	valid_0's f1: 0.863378
[282]	valid_0's binary_logloss: 0.108535	valid_0's f1: 0.863378
[283]	valid_0's binary_logloss: 0.108611	

[23]	valid_0's binary_logloss: 0.180439	valid_0's f1: 0.820173
[24]	valid_0's binary_logloss: 0.176585	valid_0's f1: 0.821556
[25]	valid_0's binary_logloss: 0.173132	valid_0's f1: 0.824899
[26]	valid_0's binary_logloss: 0.169845	valid_0's f1: 0.830645
[27]	valid_0's binary_logloss: 0.16674	valid_0's f1: 0.830986
[28]	valid_0's binary_logloss: 0.16379	valid_0's f1: 0.832665
[29]	valid_0's binary_logloss: 0.160936	valid_0's f1: 0.834748
[30]	valid_0's binary_logloss: 0.158274	valid_0's f1: 0.834913
[31]	valid_0's binary_logloss: 0.155863	valid_0's f1: 0.836816
[32]	valid_0's binary_logloss: 0.15365	valid_0's f1: 0.83714
[33]	valid_0's binary_logloss: 0.151337	valid_0's f1: 0.83887
[34]	valid_0's binary_logloss: 0.149198	valid_0's f1: 0.841168
[35]	valid_0's binary_logloss: 0.147214	valid_0's f1: 0.844028
[36]	valid_0's binary_logloss: 0.145359	valid_0's f1: 0.844904
[37]	valid_0's binary_logloss: 0.143615	valid_0's f1: 0.846343
[38]	valid_0's binary_logloss: 0.142273	valid_0's f1: 0.8473

[156]	valid_0's binary_logloss: 0.109847	valid_0's f1: 0.86553
[157]	valid_0's binary_logloss: 0.109818	valid_0's f1: 0.86553
[158]	valid_0's binary_logloss: 0.109771	valid_0's f1: 0.864993
[159]	valid_0's binary_logloss: 0.109769	valid_0's f1: 0.865275
[160]	valid_0's binary_logloss: 0.109731	valid_0's f1: 0.86594
[161]	valid_0's binary_logloss: 0.109774	valid_0's f1: 0.86594
[162]	valid_0's binary_logloss: 0.109793	valid_0's f1: 0.866477
[163]	valid_0's binary_logloss: 0.109709	valid_0's f1: 0.866477
[164]	valid_0's binary_logloss: 0.109714	valid_0's f1: 0.86553
[165]	valid_0's binary_logloss: 0.109784	valid_0's f1: 0.864583
[166]	valid_0's binary_logloss: 0.10982	valid_0's f1: 0.864045
[167]	valid_0's binary_logloss: 0.10977	valid_0's f1: 0.864045
[168]	valid_0's binary_logloss: 0.109705	valid_0's f1: 0.863507
[169]	valid_0's binary_logloss: 0.109593	valid_0's f1: 0.864045
[170]	valid_0's binary_logloss: 0.10954	valid_0's f1: 0.864045
[171]	valid_0's binary_logloss: 0.109542	valid_0

[12]	valid_0's binary_logloss: 0.243279	valid_0's f1: 0.71968
[13]	valid_0's binary_logloss: 0.235563	valid_0's f1: 0.741863
[14]	valid_0's binary_logloss: 0.228276	valid_0's f1: 0.760221
[15]	valid_0's binary_logloss: 0.221769	valid_0's f1: 0.781148
[16]	valid_0's binary_logloss: 0.21562	valid_0's f1: 0.788853
[17]	valid_0's binary_logloss: 0.209807	valid_0's f1: 0.794039
[18]	valid_0's binary_logloss: 0.204479	valid_0's f1: 0.800422
[19]	valid_0's binary_logloss: 0.199513	valid_0's f1: 0.810669
[20]	valid_0's binary_logloss: 0.19496	valid_0's f1: 0.817662
[21]	valid_0's binary_logloss: 0.191528	valid_0's f1: 0.819077
[22]	valid_0's binary_logloss: 0.18746	valid_0's f1: 0.823347
[23]	valid_0's binary_logloss: 0.183606	valid_0's f1: 0.82631
[24]	valid_0's binary_logloss: 0.17989	valid_0's f1: 0.830015
[25]	valid_0's binary_logloss: 0.176574	valid_0's f1: 0.831381
[26]	valid_0's binary_logloss: 0.17324	valid_0's f1: 0.833671
[27]	valid_0's binary_logloss: 0.170253	valid_0's f1: 0.837374

[146]	valid_0's binary_logloss: 0.112393	valid_0's f1: 0.865842
[147]	valid_0's binary_logloss: 0.112417	valid_0's f1: 0.865842
[148]	valid_0's binary_logloss: 0.112414	valid_0's f1: 0.865019
[149]	valid_0's binary_logloss: 0.112388	valid_0's f1: 0.865019
[150]	valid_0's binary_logloss: 0.112426	valid_0's f1: 0.865019
[151]	valid_0's binary_logloss: 0.112444	valid_0's f1: 0.865019
[152]	valid_0's binary_logloss: 0.112421	valid_0's f1: 0.865019
[153]	valid_0's binary_logloss: 0.112427	valid_0's f1: 0.865019
[154]	valid_0's binary_logloss: 0.112425	valid_0's f1: 0.865558
[155]	valid_0's binary_logloss: 0.112422	valid_0's f1: 0.866381
[156]	valid_0's binary_logloss: 0.112389	valid_0's f1: 0.866508
[157]	valid_0's binary_logloss: 0.112433	valid_0's f1: 0.86692
[158]	valid_0's binary_logloss: 0.112421	valid_0's f1: 0.866254
[159]	valid_0's binary_logloss: 0.112446	valid_0's f1: 0.866794
[160]	valid_0's binary_logloss: 0.112456	valid_0's f1: 0.86692
[161]	valid_0's binary_logloss: 0.112409	v

In [166]:
lgb_model.feature_importance()

array([255, 445,  99, 148,  16, 110,  78,  87,  88,  50, 134, 125,  17,
         8, 105,  24,  26,  21,   5,  90, 127,   2, 140, 144,  14,  59,
        11,  72, 205,   3,  23, 168, 161, 240, 105,   2,   1,  27,  16,
        16,   2, 539, 125, 107, 137,  85, 235, 129,  57,   6,  88,  54,
        55,  82,  52, 264,  63,  51,  24,  75,  32,  26,  69,  18,  20,
        43,  50,  29,   4, 113,  11,  28,  59,  66,  18,   4,  63,   4,
        11,  69,  34,  13,   4,  43,   6,   7,  34,  57,  55,  18,  70,
        62,  51,  88,  50,   5,   4,  67,   0,   2,  63,  53,   1,   1,
        83,   0,   0,  51,  11, 114,   3,  41])

In [168]:
from sklearn.metrics import roc_auc_score
df_oof = pd.concat(oof)
df_oof.sort_values(['pred'], inplace=True, ascending=False)
df_oof.reset_index(drop=True, inplace=True)
df_oof['pred_label'] = 0
df_oof.loc[:int(0.15 * len(df_oof)), 'pred_label'] = 1
f1 = f1_score(df_oof["标签"], df_oof['pred_label'])
auc = roc_auc_score(df_oof["标签"], df_oof['pred'])
f1, auc

(0.8659642693540999, 0.9799982995060352)

In [184]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby([
    'column'
])['importance'].agg('mean')#.sort_values(ascending=False).reset_index()
df_importance.to_csv(r'result_import.csv')

In [ ]:
# 112维特征：0.8660  ；0.8692

## online

In [52]:
train_data.head()

,Unnamed: 0,岗位编号,求职者编号,标签
0,0,775040,5969825,0
1,1,815337,6212884,0
2,2,770107,5451198,0
3,3,43795661,320873919,1
4,4,798922,5650936,0


In [176]:

# online
data=pd.concat([train_data,valid_data],ignore_index=True)
train=get_k_fold_data(data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)

temp_data= pd.concat([train, test], ignore_index=True)
# data_all_feature=feature_process(temp_data)
temp_data=deliver_record_feature(temp_data)
for f in temp_data.select_dtypes('object'):
    le = LabelEncoder()
    print(f)
    temp_data[f] = le.fit_transform(temp_data[f].astype('str'))
temp_data["招聘对象"] =le.fit_transform(temp_data["招聘对象"].astype('str'))
temp_data["最近工作岗位"] =le.fit_transform(temp_data["最近工作岗位"].astype('str'))
temp_data["求职意向岗位类别"] =le.fit_transform(temp_data["求职意向岗位类别"].astype('str'))





100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:07<00:00,  1.10s/it]


应聘者专业
最近工作岗位
最近所在行业
当前工作所在地
语言能力
专业特长
招聘职位
对应聘者的专业要求
岗位工作地点
具体要求
自荐信
求职意向工作地点
求职意向所在行业
其他说明
汉语语言能力
英语语言能力
粤语语言能力
日语语言能力


In [177]:
temp_data.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [178]:
temp_data.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [179]:
def lgb_model_for_upload(data, label):
    model = lightgbm.train(train_set=lightgbm.Dataset(data.values, label.values),
                           num_boost_round=800,
                           params=params)

    return model

In [180]:
data_all_feature  =temp_data
#train,train_label,valid,valid_label
train0 = data_all_feature[:train.shape[0]]#.标签.value_counts()
test0 = data_all_feature[train.shape[0]:]
train_label = train0["标签"]
train=train0[list(train0.columns)[:2]+list(train0.columns)[3:]]


In [181]:
model = lgb_model_for_upload(train, train_label)

In [182]:
# from utils import submission
def submission(path, model, test0, name):
    test0["预测打分"] = model.predict(test0[list(test0.columns)[:2]+list(test0.columns)[3:]].values)
    test_data = test0.sort_values("预测打分", ascending=False, ignore_index=True)
    test_data["预测"] = 0
    test_data.loc[:int(0.15 * len(test_data)), ["预测"]] = 1

    submit = test_data[["岗位编号", "求职者编号", "预测"]]
    submit.columns = ["RECRUIT_ID", "PERSON_ID", "LABEL"]
    submit.to_csv(path + name + ".csv", index=False)
path2=path0+'/result/'
name ='online_112_feature'
submission(path2,model,test0,name)

d:\software\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [183]:
data_all_feature.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [86]:
test0

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate,预测打分
0,135144,996499,0,1,15,1.0,80,34,226,0,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.943105
1,135144,3301030,0,1,12,2.0,112,37,287,0,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.989315
2,135144,3404860,0,1,0,2.0,0,32,1,60,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.714051
3,135144,4155520,0,1,11,3.0,0,32,1,60,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.859480
4,135144,4211720,0,1,12,1.0,211,30,226,0,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.602499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70769,44350657,321173912,0,1,5,4.0,377,26,198,38,...,7.0,56.30374,7.0,7.0,0.555556,1,0.888889,1.0,1.00,0.999980
70770,44350657,321377962,0,1,0,4.0,292,28,226,0,...,7.0,56.30374,7.0,7.0,0.555556,1,0.888889,1.0,1.00,0.999992
70771,47578782,319583916,0,0,0,4.0,302,28,226,0,...,7.0,NaN,7.0,7.0,1.000000,0,1.000000,0.0,1.00,0.999959
70772,47649893,6079047,0,1,7,4.0,0,29,174,82,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,0.0,1.00,0.997052


* one-hot +岗位编号	求职者编号 0.8633      ONLINE:0.8653
* no one hot +岗位编号	求职者编号 0.865602      ONLINE:0.8663
* 添加招聘岗位投递记录中的求职意向岗位，求职者专业，（年龄、工作经历数、项目经验数、可到职天数）统计指标、工作地点符合占比特征
* 0.8665 online：0.86748
* 添加k-折交叉的标签特征：offline:0.8772  online: 0.8711
* 112维特征：0.869288

In [2]:
import pandas as pd
df1 = pd.DataFrame([1,2,3,4,5,6])

In [5]:
df1[1]=[1,1,1,1,2,2]

In [6]:
df1

,0,1
0,1,1
1,2,1
2,3,1
3,4,1
4,5,2
5,6,2


In [7]:
f1=1
f2=0
df1.groupby(f1)[f2].agg(
        [(f"{f1}_{f2}_mean", "mean"),
         (f"{f1}_{f2}_max", "max"),
         (f"{f1}_{f2}_min", "min"),
         (f"{f1}_{f2}_std", "std")]
    ).reset_index()

,1,1_0_mean,1_0_max,1_0_min,1_0_std
0,1,2.5,4,1,1.290994
1,2,5.5,6,5,0.707107
